In [59]:
%gui qt
%matplotlib qt
import xmcd_projection
from xmcd_projection import deep_reload
deep_reload(xmcd_projection)
from xmcd_projection import *

### Summary how to do it:
- use only the outside faces for projection and for each of those faces, find the centroid
- get the intersection points: face_ray_intersection(p, pt, triangles_pt) where pt are centroids
- displace the triangles by the points on faces and check if origin in triangle: origin_in_triangle(triangles_pt - points_on_faces[:,np.newaxis,:])
- extract the triangles that are pierced by the beam and calculate the distances between their intersections
- for each tetra calulate the average magnetisation
- calculate the attenuation of the beam for each tetra and sum over the attenuations for all pierced tetra

In [ ]:

structure_file = r"C:\Users\lukas\OneDrive - University of Cambridge\PhD\Simulations\spirals\models\spiral_det_in_50w30t.msh"
magnetisation_file = r"F:\PhD_data\Simulations\spiral_automotion_simulations\data\basic_automotion\spiral_det_in_50w30t_005\data\data.30.csv"

mesh = get_mesh(structure_file)
mesh = ground_mesh(mesh)
magnetisation, _ = get_magnumfe_magnetisation(magnetisation_file)

### Getting the projected structure and the piercings takes long

In [ ]:
struct_projected, piercings_list, p = get_projection_with_piercings(mesh)

### However, once we have the piercings, it should be quick to get the xmcd

In [5]:
xmcd_value = get_xmcd_from_piercings_list(mesh, magnetisation, piercings_list)

### Display and save render

In [6]:
struct = get_struct_from_mesh(mesh)
mag_colors = get_struct_face_mag_color(struct, magnetisation)

vis = MagnumfeVisualizer(struct, struct_projected, xmcd_value, struct_colors=mag_colors)
vis.show()

In [7]:
vis.set_camera(dist=1e5, center=[0,-800,0])

In [10]:
vis.save_render('./img/magnumfe_demo.png', size=(1024, 1024))

### Save the data

In [ ]:
save_file = 'test_saving.npy'
save_piercing_data(save_file, struct, struct_projected, piercings_list, p, mesh)

#### For loading

In [4]:
load_file = 'test_saving.npy'
struct, struct_projected, piercings_list, p, mesh = load_piercing_data(load_file)

## Step by step what I am doing

#### Show projection

In [47]:
ax = show_structure_points(struct)

show_structure_points(struct_projected, ax=ax)

ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('z')

Text(0.5, 0, 'z')

#### Take a point and show a pierced ray

In [48]:
projected_points = np.array(struct_projected.vertices)
_, _, tetra = get_mesh_data(mesh)

i = 1000
pt = projected_points[struct_projected.faces[i,0], :]

phi, theta = 90, 15
theta_r = np.deg2rad(theta)
phi_r = np.deg2rad(phi)

nx0 = np.cross(p, pt)
y = np.linspace(-1000, 0, 1000)
x = nx0[1]/np.sin(theta_r)*np.ones(y.shape)
z = y*np.tan(theta_r)+nx0[0]/np.cos(theta_r)

ax.plot(x, y, z, c='red')

ax.scatter(pt[0], pt[1], pt[2], s=100, c='g')

# get the points of the pierced tetrahedra
indx = piercings_list[i][1]
points = np.array(struct.vertices)
pierced_tetra_points = points[tetra[indx, :].flatten(), :]

ax.scatter(pierced_tetra_points[:,0], pierced_tetra_points[:,1], pierced_tetra_points[:,2], s=100, c='g')

In [49]:
plt.savefig('img/piercing_demo.png', dpi=300)

#### Show one of the pierced tetrahedra

In [64]:
from mpl_toolkits.mplot3d import Axes3D

deep_reload(xmcd_projection)
from xmcd_projection import *

# Create a new plot
fig = plt.figure()
ax = fig.add_subplot((111), projection='3d', proj_type='ortho')

selected_triangles = get_triangles_from_tetra(points, tetra[indx, :])
vts = selected_triangles.reshape(-1, 3)
ax.plot_trisurf(vts[:,0], vts[:,1], vts[:,2])
ax.scatter(vts[:,0], vts[:,1], vts[:,2], 'ko')
ax.set_xlabel('x')


y = np.linspace(-150, -80, 100)
x = nx0[1]/np.sin(theta_r)*np.ones(y.shape)
z = y*np.tan(theta_r)+nx0[0]/np.cos(theta_r)

ax.plot(x, y, z, c='red')

In [65]:
plt.savefig('img/piercing_demo1.png', dpi=300)